## Signal Reconstruction and Lattent space
In this notebook we will play with signal reconstruction and lattent space variables

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
from ipywidgets import interactive
import numpy as np
import mxnet.ndarray as nd
  
from mlss_gdansk2019.SignalGenerator import SignalGenerator
from mlss_gdansk2019.SinBayesianAutoEncoderModel import SinBayesianAutoEncoder
from mlss_gdansk2019 import plot_utils 

np.random.seed(12234)

### First we load Bayesian autoencoder we have trained

In [ ]:
sin_bae = SinBayesianAutoEncoder()
sin_bae.load_parameters("model/model.params")

### Lets generate signal with 0.8 HZ frequency

In [ ]:
signal = np.sin(np.arange(0, 50) * 0.16 * np.pi)

### Pass the signal through the network and reconstruct it. 

We will set teacher_forcing_prob to 0, since during infeerence we do not know what was the actual signal we want to infer. For now we wouldn't override latent space value for signal predicted by the model 

In [ ]:
teacher_forcing_prob = 0
latent_space_override = None
auto_encoder_output = sin_bae(nd.array([signal]), teacher_forcing_prob, latent_space_override)

estimated_latent_space_vals = auto_encoder_output.latent_space_val[0].asnumpy()
reconstructed_signal = auto_encoder_output.decoded_signal[0].asnumpy()

In [ ]:
plot_utils.plot_signals(signal, reconstructed_signal, estimated_latent_space_vals)

As you see in terms of frequencies both signals overlap very well, and this is what we have expected.

### Now we can try to modify lattent space varialbe to reconstruct the signal

In [ ]:
teacher_forcing_prob = 0
latent_space_override = nd.array([-1.5, -1.5]).reshape(1,2)
auto_encoder_output = sin_bae(nd.array([signal]), teacher_forcing_prob, latent_space_override)

estimated_latent_space_vals = auto_encoder_output.latent_space_val[0].asnumpy()
reconstructed_signal = auto_encoder_output.decoded_signal[0].asnumpy()

plot_utils.plot_signals(signal, reconstructed_signal, estimated_latent_space_vals)

### Lets now process more signalss through the network and visualize the latent space

In [ ]:
signal_generator = SignalGenerator()
frequencies, phase, signals = signal_generator.generate_signals(length=50, num=200)
signal_generator.plot_signals_with_multipliers(frequencies, phase, signals)

### Process signals through the network and visualize predicted lattent space values

Lets take a look how does Gaussian latent space looks for signals we have processed through the model 

In [ ]:
teacher_forcing_prob = 0
latent_space_override = None
auto_encoder_output = sin_bae(nd.array(signals), teacher_forcing_prob, latent_space_override)
latent_space = auto_encoder_output.latent_space_val.asnumpy()
plot_utils.plot_histograms(latent_space)

In [ ]:
plot_utils.plot_latent_space(latent_space=latent_space, phase=phase, frequencies=frequencies)

This is entangled!!

Play on your own to see how model behaves dependently on latent space value.

In [ ]:
def f(lv_1, lv_2):
    teacher_forcing_prob = 0
    latent_space_override = nd.array([lv_1, lv_2]).reshape(1, 2)
    auto_encoder_output = sin_bae(nd.array(signals),
                                  teacher_forcing_prob,
                                  latent_space_override)
    signal = signals[0, :]
    latent_space = auto_encoder_output.latent_space_val[0].asnumpy()
    decoded_signal = auto_encoder_output.decoded_signal[0, :].asnumpy()
    plot_utils.plot_signals(signal, decoded_signal, latent_space)
    

In [ ]:
interactive_plot = interactive(f, lv_1=(-2., 2.), lv_2=(-2., 2.))
output = interactive_plot.children[-1]
output.layout.height = '350px'
interactive_plot